In [ ]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
from scipy.stats import iqr
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

Mounted at /content/drive


In [ ]:
# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')
# 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)
# Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# 폰트 설정

Selecting previously unselected package fonts-nanum.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data

In [ ]:
mountInstance = LoadGoogleDriveData()

### 1.오아시스 비즈니스(필지고유번호 & 법정동읍면동코드 & 외식업종)  데이터

In [ ]:
CONTEST_2023 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "소상공인 매출등급 예상 대상 필지.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

In [ ]:
CONTEST_2022 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "필지단위 소상공인 매출등급 정보.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

### 2. KCD(상권코드 & 외식업종 중분류) 소상공인 신용데이터

In [ ]:
KCD_2022_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "SOHO_MERGE_DATA_LIST.csv",
    unicode = "utf-8", columnTF = False)

### 3. 행정동코드 - 상권코드 매핑 데이터

In [ ]:
MARKET_LOC_MAP = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/",
    file_name_extension = "market_loc_map.csv",
    unicode = "euc-kr", columnTF = False)

In [ ]:
MARKET_LOC_MAP

,상권_코드_2022,상권_코드_2023,행정동_코드,법정동_코드,STDG_EMD_CD
0,2110008,3110008,11110515,11110101,11110101
1,2110008,3110008,11110515,11110102,11110102
2,2110008,3110008,11110515,11110103,11110103
3,2110008,3110008,11110515,11110104,11110104
4,2110008,3110008,11110515,11110105,11110105
...,...,...,...,...,...
3453,2130102,3130102,11230750,11230110,11230110
3454,2130166,3130166,11380590,11380107,11380107
3455,2130113,3130113,11260690,11260106,11260106
3456,2130124,3130124,11290780,11290138,11290138


In [ ]:
# MARKET_LOC_MAP.drop(["상권_코드_2022", "상권_코드_2023"], axis = 1, inplace = True)

### 4. 서울열린데이터광장 상권특성 공공데이터

In [ ]:
MERGE_2022_DATA = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_RAW_2022_DATA.csv",
    unicode = "utf-8", columnTF = False).drop_duplicates()
# MERGE_2022_NAZERO_DATA = MERGE_2022_DATA.fillna(0)

<ipython-input-5-322a33626855>:7: DtypeWarning: Columns (26,27,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(os.path.join(file_path + file_name_extension),


In [ ]:
print(len(MERGE_2022_DATA), len(MERGE_2022_DATA[MERGE_2022_DATA["상권_코드_명"].isna() == False]))

192718 192718


In [ ]:
MERGE_2023_DATA = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_RAW_2023_DATA.csv",
    unicode = "utf-8", columnTF = False).drop_duplicates()
# MERGE_2023_NAZERO_DATA = MERGE_2023_DATA.fillna(0)

<ipython-input-5-322a33626855>:7: DtypeWarning: Columns (5,26,27,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(os.path.join(file_path + file_name_extension),


In [ ]:
MERGE_2023_DATA = MERGE_2023_DATA[MERGE_2023_DATA["상권_코드_명"].isna() == False]
MERGE_2023_DATA

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,...,기준_월_코드,법정동_코드,STDG_EMD_CD,행정동_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2023,1,A,골목상권,3110001,이북5도청사,NaN,NaN,520427635.0000,129126028.0000,...,12.0000,11110182.0000,11110182.0000,평창동,HH,정체,111.0000,58.0000,104.0000,52.0000
1,2023,1,A,골목상권,3110001,이북5도청사,NaN,NaN,520427635.0000,129126028.0000,...,12.0000,11110183.0000,11110183.0000,평창동,HH,정체,111.0000,58.0000,104.0000,52.0000
6,2023,1,A,골목상권,3110002,독립문역 1번,NaN,NaN,232465205.0000,54998987.0000,...,12.0000,11110187.0000,11110187.0000,무악동,HH,정체,105.0000,64.0000,104.0000,52.0000
8,2023,1,A,골목상권,3110003,세검정초등학교,NaN,NaN,1349699230.0000,336704220.0000,...,12.0000,11110184.0000,11110184.0000,부암동,LH,상권확장,99.0000,56.0000,104.0000,52.0000
9,2023,1,A,골목상권,3110003,세검정초등학교,NaN,NaN,1349699230.0000,336704220.0000,...,12.0000,11110185.0000,11110185.0000,부암동,LH,상권확장,99.0000,56.0000,104.0000,52.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141694,2023,1,R,전통시장,3130154,상계주공1단지(가)상가,NaN,NaN,NaN,NaN,...,12.0000,11350105.0000,11350105.0000,상계6?7동,LL,다이나믹,100.0000,48.0000,104.0000,52.0000
141696,2023,1,R,전통시장,3130155,시영B상가(중계 그린아파트 B 상가),NaN,NaN,NaN,NaN,...,12.0000,11350106.0000,11350106.0000,중계2?3동,LH,상권확장,97.0000,52.0000,104.0000,52.0000
141699,2023,1,R,전통시장,3130156,시영2단지 무지개종합상가(중계 무지개2단지아파트상가),NaN,NaN,NaN,NaN,...,12.0000,11350106.0000,11350106.0000,중계2?3동,LH,상권확장,97.0000,52.0000,104.0000,52.0000
141702,2023,1,D,발달상권,3120098,DMC(디지털미디어시티),NaN,NaN,NaN,NaN,...,12.0000,11440127.0000,11440127.0000,상암동,LL,다이나믹,75.0000,42.0000,104.0000,52.0000


In [ ]:
"""
# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2022"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2022 = os.listdir(file_path)
print(directory_상권_2022, end = ", ")

data_상권특성_2022: List = [
    (directory_상권_2022[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2022))
]
print("\n", data_상권특성_2022, end = ", ")

# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2023"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2023 = os.listdir(file_path)
print(directory_상권_2023, end = ",")

data_상권특성_2023: List = [
    (directory_상권_2023[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2023))
]
print("\n", data_상권특성_2023, end = ", ")

# 상권특성_2022 및 상권특성_2023 데이터의 교집합
print(np.intersect1d(data_상권특성_2022, data_상권특성_2023))
# 상권특성_2022 및 상권특성_2023 데이터의 여집합
print(np.setxor1d(data_상권특성_2022, data_상권특성_2023))
directory_상권_2022.remove("서울시 상권분석서비스(상권배후지-소득소비).csv")
data_상권특성_2022.remove("상권배후지-소득소비")

MARKET_2022_LIST = []
for index, data in zip(list(range(len(directory_상권_2022))), directory_상권_2022):
  MARKET_2022 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2022/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2022_LIST.append(MARKET_2022)
  MARKET_2022 = 0
MARKET_2023_LIST = []
directory_상권_2023: List = [item.split(".csv")[0] + "_2023.csv" for item in directory_상권_2022[1:]]

for index, data in zip(list(range(len(directory_상권_2023))), directory_상권_2023):
  MARKET_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2023/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2023_LIST.append(MARKET_2023)
  MARKET_2023 = 0

display(directory_상권_2022)
# MARKET_2022_LIST[0]
"""

In [ ]:
"""
MARKET_2022_LIST[11].rename(columns = {" 상권_구분_코드_명":"상권_구분_코드_명"}, inplace = True)
for index in [2, 3, 6, 7, 11]:
  if index == 2:
    MERGE_2022_DATA = MARKET_2022_LIST[index]
  else:
    MERGE_2022_DATA = pd.merge(
        left = MERGE_2022_DATA, right = MARKET_2022_LIST[index],
        on = ["기준_년_코드", "기준_분기_코드", "상권_구분_코드", "상권_구분_코드_명", "상권_코드", "상권_코드_명"],
        how = "inner")
MARKET_2022_LIST[0].drop(["형태정보"], axis = 1, inplace = True)
MARKET_LOC_MAP.info()
MARKET_2022_LIST[13] = pd.merge(
    left = MARKET_LOC_MAP, right = MARKET_2022_LIST[13],
    on = ["행정동_코드"],
    how = "right")
# MARKET_2022_LIST[13].rename(columns = {"상권_코드_2022":"상권_코드"}, inplace = True)
# MARKET_2022_LIST[13].drop(["상권_코드_2022", "상권_코드_2023"], axis = 1, inplace = True)

MARKET_2022_LIST[0]["기준_년_코드"] = MARKET_2022_LIST[0]["기준_년월_코드"].apply(lambda data: str(data)[0:4])
MARKET_2022_LIST[0]["기준_월_코드"] = MARKET_2022_LIST[0]["기준_년월_코드"].apply(lambda data: str(data)[4:6])

# MERGE_2022_DATA2 = pd.merge(
      # left = MARKET_2022_LIST[0][["상권_구분_코드", "상권_코드_명", "엑스좌표_값", "와이좌표_값", "시군구_코드", "행정동_코드"]], right = MARKET_2022_LIST[13],
      # on = ["기준_년_코드", "기준_분기_코드", "행정동_코드"],
      # how = "inner")

MARKET_2022_LIST[0] = MARKET_2022_LIST[0].astype({"기준_년_코드":"int64", "기준_월_코드":"int64"})
MERGE_2022_DATA2 = pd.merge(
        left = MARKET_2022_LIST[0], right = MARKET_2022_LIST[13],
        on = ["기준_년_코드", "행정동_코드"],
        how = "inner")

MERGE_2022_DATA = pd.merge(
        left = MERGE_2022_DATA, right = MERGE_2022_DATA2,
        on = ["기준_년_코드", "기준_분기_코드", "상권_구분_코드", "상권_구분_코드_명", "상권_코드", "상권_코드_명"],
        how = "outer")
pd.set_option("display.max_column", None)
MERGE_2022_DATA.info()
MERGE_2022_DATA.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_MERGE_2022_DATA.csv", index = False)
"""

In [ ]:
"""
MARKET_2022_LIST[0]["기준_년_코드"] = MARKET_2022_LIST[0]["기준_년월_코드"].apply(lambda data: str(data)[0:4])
MARKET_2022_LIST[0]["기준_월_코드"] = MARKET_2022_LIST[0]["기준_년월_코드"].apply(lambda data: str(data)[4:6])
MARKET_2022_LIST[0] = MARKET_2022_LIST[0].astype({"기준_년_코드":"int64", "기준_월_코드":"int64"})

MARKET_2023_LIST[10].rename(columns = {" 상권_구분_코드_명":"상권_구분_코드_명"}, inplace = True)
for index in [1, 2, 5, 6, 10]:
  if index == 1:
    MERGE_2023_DATA = MARKET_2023_LIST[index]
  else:
    MERGE_2023_DATA = pd.merge(
        left = MERGE_2023_DATA, right = MARKET_2023_LIST[index],
        on = ["기준_년_코드", "기준_분기_코드", "상권_구분_코드", "상권_구분_코드_명", "상권_코드", "상권_코드_명"],
        how = "outer")
MARKET_2023_LIST[12] = pd.merge(
    left = MARKET_LOC_MAP, right = MARKET_2023_LIST[12],
    on = ["행정동_코드"],
    how = "inner")
MERGE_2023_DATA.info()
MARKET_2023_LIST[12].info()
# MARKET_2023_LIST[12].rename(columns = {"상권_코드_2023":"상권_코드"}, inplace = True)
# MARKET_2023_LIST[12].drop(["상권_코드_2022", "상권_코드_2023"], axis = 1, inplace = True)

MERGE_2023_DATA2 = pd.merge(
        left = MARKET_2022_LIST[0].drop("기준_년_코드", axis = 1), right = MARKET_2023_LIST[12],
        on = ["행정동_코드"],
        how = "inner")

MERGE_2023_DATA = pd.merge(
        left = MERGE_2023_DATA, right = MERGE_2023_DATA2,
        on = ["기준_년_코드", "기준_분기_코드", "상권_구분_코드", "상권_구분_코드_명", "상권_코드", "상권_코드_명"],
        how = "outer")
pd.set_option("display.max_column", None)
MERGE_2023_DATA.info()
MERGE_2023_DATA.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_MERGE_2023_DATA.csv", index = False)
"""

In [ ]:
"""
print(sum(MERGE_2022_NAZERO_DATA.duplicated()), sum(MERGE_2023_NAZERO_DATA.duplicated()))
  # 77488 57763
MERGE_2022_UNIQUE_DATA = MERGE_2022_NAZERO_DATA.drop_duplicates()
  # pd.dataFrame.T.duplicated().T
MERGE_2023_UNIQUE_DATA = MERGE_2023_NAZERO_DATA.drop_duplicates()
print(sum(MERGE_2022_UNIQUE_DATA.duplicated()), sum(MERGE_2023_UNIQUE_DATA.duplicated()))
  # 0 0
MERGE_2022_UNIQUE_DATA.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_MERGE_2022_DATA.csv", index = False)
MERGE_2023_UNIQUE_DATA.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_MERGE_2023_DATA.csv", index = False)
"""

### 5.분기별 상권특성 공공데이터 전처리

#### 5.(1) 상권_소득소비

  MARKET_2022_LIST[2]

  MARKET_2023_LIST[1]

  ```
  SELECT
    AVERAGE(seoul.상권특성.교육_지출_총금액),
    AVERAGE(seoul.상권특성.교통_지출_총금액),
    AVERAGE(seoul.상권특성.생활용품_지출_총금액),
    AVERAGE(seoul.상권특성.문화_지출_총금액),
    AVERAGE(seoul.상권특성.유흥_지출_총금액),
    AVERAGE(seoul.상권특성.의료비_지출_총금액),
    AVERAGE(seoul.상권특성.의류_신발_지출_총금액),
    AVERAGE(seoul.상권특성.식료품_지출_총금액),
    AVERAGE(seoul.상권특성.여가_지출_총금액),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [ ]:
# 외식업종만 찾아서 행 추출
# '커피-음료', '호프-간이주점', '분식전문점','일식음식점', '한식음식점', '치킨전문점', '제과점', '양식음식점', '패스트푸드점'
# MERGE_2022_NAZERO_DATA["상권_구분_코드_명"].unique()
# MERGE_2022_NAZERO_DATA["서비스_업종_코드_명"].unique()
# 외식업종_행번호 = MERGE_2022_UNIQUE_DATA["서비스_업종_코드_명"].isin(['커피-음료', '호프-간이주점', '분식전문점','일식음식점', '한식음식점', '치킨전문점', '제과점', '양식음식점', '패스트푸드점'])
# MERGE_2022_UNIQUE_DATA = MERGE_2022_UNIQUE_DATA[외식업종_행번호]
# 192718행 > 28253행

In [ ]:
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
display(MERGE_2022_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["지출_총금액",
       "식료품_지출_총금액",
       "의류_신발_지출_총금액",
       "생활용품_지출_총금액",
       "의료비_지출_총금액",
       "교통_지출_총금액",
       "여가_지출_총금액",
       "문화_지출_총금액",
       "교육_지출_총금액",
       "유흥_지출_총금액"]].agg(["mean"]))
  # "서비스_업종_코드_명" 인덱스컬럼에 추가해야 함(X) > Error

지출_총금액  \
                                                                 mean   
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                        
2022    1        2110001 이북5도청사                        520427635.0000   
                 2110002 독립문역 1번                       232465205.0000   
                 2110003 세검정초등학교                      1349699230.0000   
                 2110004 대신고등학교                        712772785.0000   
                 2110005 세검정                           884701599.0000   
...                                                               ...   
        4        3130323 둔촌역전통시장                               0.0000   
                 3130324 길동복조리시장                               0.0000   
                 3130325 명일전통시장                                0.0000   
                 3130326 고덕 골목형상점가                             0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)          0.0000   

                                                          식료품_지출_총금액  \
                                                                mean   
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2022    1        2110001 이북5도청사                       129126028.0000   
                 2110002 독립문역 1번                       54998987.0000   
                 2110003 세검정초등학교                      336704220.0000   
                 2110004 대신고등학교                       205649728.0000   
                 2110005 세검정                          221446948.0000   
...                                                              ...   
        4        3130323 둔촌역전통시장                              0.0000   
                 3130324 길동복조리시장                              0.0000   
                 3130325 명일전통시장                               0.0000   
                 3130326 고덕 골목형상점가                            0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)         0.0000   

                                                        의류_신발_지출_총금액  \
                                                                mean   
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2022    1        2110001 이북5도청사                        64853632.0000   
                 2110002 독립문역 1번                       28932090.0000   
                 2110003 세검정초등학교                      166150296.0000   
                 2110004 대신고등학교                        84170011.0000   
                 2110005 세검정                          109882417.0000   
...                                                              ...   
        4        3130323 둔촌역전통시장                              0.0000   
                 3130324 길동복조리시장                              0.0000   
                 3130325 명일전통시장                               0.0000   
                 3130326 고덕 골목형상점가                            0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)         0.0000   

                                                        생활용품_지출_총금액  \
                                                               mean   
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                      
2022    1        2110001 이북5도청사                       37122367.0000   
                 2110002 독립문역 1번                      17084860.0000   
                 2110003 세검정초등학교                      98720958.0000   
                 2110004 대신고등학교                       51668351.0000   
                 2110005 세검정                          64738346.0000   
...                                                             ...   
        4        3130323 둔촌역전통시장                             0.0000   
                 3130324 길동복조리시장                             0.0000   
                 3130325 명일전통시장                              0.0000   
                 3130326 고덕 골목형상점가                           0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)        0.0000   

                           

In [ ]:
display(MERGE_2023_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["지출_총금액",
       "식료품_지출_총금액",
       "의류_신발_지출_총금액",
       "생활용품_지출_총금액",
       "의료비_지출_총금액",
       "교통_지출_총금액",
       "여가_지출_총금액",
       "문화_지출_총금액",
       "교육_지출_총금액",
       "유흥_지출_총금액"]].agg("mean"))

지출_총금액  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2023    1        3001491 이태원 관광특구                             0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구               0.0000   
                 3001493 동대문패션타운 관광특구                         0.0000   
                 3001494 종로?청계 관광특구                           0.0000   
                 3001495 잠실 관광특구                              0.0000   
...                                                              ...   
                 3130323 둔촌역전통시장                      764962470.0000   
                 3130324 길동복조리시장                      595074220.0000   
                 3130325 명일전통시장                       762846136.0000   
                 3130326 고덕 골목형상점가                     40697223.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)         0.0000   

                                                          식료품_지출_총금액  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2023    1        3001491 이태원 관광특구                             0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구               0.0000   
                 3001493 동대문패션타운 관광특구                         0.0000   
                 3001494 종로?청계 관광특구                           0.0000   
                 3001495 잠실 관광특구                              0.0000   
...                                                              ...   
                 3130323 둔촌역전통시장                      212600940.0000   
                 3130324 길동복조리시장                      151696675.0000   
                 3130325 명일전통시장                       200045982.0000   
                 3130326 고덕 골목형상점가                     11206611.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)         0.0000   

                                                       의류_신발_지출_총금액  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                      
2023    1        3001491 이태원 관광특구                            0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구              0.0000   
                 3001493 동대문패션타운 관광특구                        0.0000   
                 3001494 종로?청계 관광특구                          0.0000   
                 3001495 잠실 관광특구                             0.0000   
...                                                             ...   
                 3130323 둔촌역전통시장                      92152548.0000   
                 3130324 길동복조리시장                      73524317.0000   
                 3130325 명일전통시장                       92132092.0000   
                 3130326 고덕 골목형상점가                     4772763.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)        0.0000   

                                                        생활용품_지출_총금액  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                      
2023    1        3001491 이태원 관광특구                            0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구              0.0000   
                 3001493 동대문패션타운 관광특구                        0.0000   
                 3001494 종로?청계 관광특구                          0.0000   
                 3001495 잠실 관광특구                             0.0000   
...                                                             ...   
                 3130323 둔촌역전통시장                      54122662.0000   
                 3130324 길동복조리시장                      44501832.0000   
                 3130325 명일전통시장                       50249063.0000   
                 3130326 고덕 골목형상점가                     2901471.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)        0.0000   

                                                          의료비_지출_총금액  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2023    1        3001491 이태원 관광특구                             0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구               0.0000   
                 3001493 동대문패션타운 관광특구               

In [ ]:
print(len(MERGE_2022_DATA["상권_코드_명"].unique()), len(MERGE_2023_DATA["상권_코드_명"].unique()))

1650 1650


In [ ]:
np.setxor1d(MERGE_2022_DATA["상권_코드_명"].unique().tolist(), MERGE_2022_DATA["상권_코드_명"].unique().tolist())

array([], dtype='<U32')

#### 5.(2) 상권_배후지_아파트
  
  MARKET_2022_LIST[3]

  MARKET_2023_LIST[2]

  ```
  SELECT
    AVERAGE(seoul.상권특성.아파트_단지_수),
    AVERAGE(seoul.상권특성.아파트_가격_1_억_미만_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_1_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_2_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_3_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_4_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_5_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_6_억_이상_세대_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [ ]:
display(MERGE_2022_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["아파트_단지_수",
       "아파트_가격_1_억_미만_세대_수",
       "아파트_가격_1_억_세대_수",
       "아파트_가격_2_억_세대_수",
       "아파트_가격_3_억_세대_수",
       "아파트_가격_4_억_세대_수",
       "아파트_가격_5_억_세대_수",
       "아파트_가격_6_억_이상_세대_수"]].agg("mean"))
  # ["mean", "max", "min"]

아파트_단지_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                  
2022    1        2110001 이북5도청사                         21.0000   
                 2110002 독립문역 1번                         7.0000   
                 2110003 세검정초등학교                        70.0000   
                 2110004 대신고등학교                          6.0000   
                 2110005 세검정                            40.0000   
...                                                         ...   
        4        3130323 둔촌역전통시장                         0.0000   
                 3130324 길동복조리시장                         0.0000   
                 3130325 명일전통시장                          0.0000   
                 3130326 고덕 골목형상점가                       0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)    0.0000   

                                                       아파트_가격_1_억_미만_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                            
2022    1        2110001 이북5도청사                                   11.0000   
                 2110002 독립문역 1번                                   4.0000   
                 2110003 세검정초등학교                                  30.0000   
                 2110004 대신고등학교                                   11.0000   
                 2110005 세검정                                      51.0000   
...                                                                   ...   
        4        3130323 둔촌역전통시장                                   0.0000   
                 3130324 길동복조리시장                                   0.0000   
                 3130325 명일전통시장                                    0.0000   
                 3130326 고덕 골목형상점가                                 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)              0.0000   

                                                       아파트_가격_1_억_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                         
2022    1        2110001 이북5도청사                                 4.0000   
                 2110002 독립문역 1번                               13.0000   
                 2110003 세검정초등학교                              313.0000   
                 2110004 대신고등학교                                 6.0000   
                 2110005 세검정                                  101.0000   
...                                                                ...   
        4        3130323 둔촌역전통시장                                0.0000   
                 3130324 길동복조리시장                                0.0000   
                 3130325 명일전통시장                                 0.0000   
                 3130326 고덕 골목형상점가                              0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)           0.0000   

                                                       아파트_가격_2_억_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                         
2022    1        2110001 이북5도청사                                35.0000   
                 2110002 독립문역 1번                               27.0000   
                 2110003 세검정초등학교                              224.0000   
                 2110004 대신고등학교                                11.0000   
                 2110005 세검정                                  159.0000   
...                                                                ...   
        4        3130323 둔촌역전통시장                                0.0000   
                 3130324 길동복조리시장                                0.0000   
                 3130325 명일전통시장                                 0.0000   
                 3130326 고덕 골목형상점가                              0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)           0.0000   

                                                       아파트_가격_3_억_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                         
2022    1        2110001 이북5도청사                               105.0000   
                 2110002 독립문역 1번 

In [ ]:
display(MERGE_2023_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["아파트_단지_수",
       "아파트_가격_1_억_미만_세대_수",
       "아파트_가격_1_억_세대_수",
       "아파트_가격_2_억_세대_수",
       "아파트_가격_3_억_세대_수",
       "아파트_가격_4_억_세대_수",
       "아파트_가격_5_억_세대_수",
       "아파트_가격_6_억_이상_세대_수"]].agg("mean"))

아파트_단지_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                  
2023    1        3001491 이태원 관광특구                       53.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구          0.0000   
                 3001493 동대문패션타운 관광특구                    2.0000   
                 3001494 종로?청계 관광특구                      3.0000   
                 3001495 잠실 관광특구                        27.0000   
...                                                         ...   
                 3130323 둔촌역전통시장                         0.0000   
                 3130324 길동복조리시장                         0.0000   
                 3130325 명일전통시장                          0.0000   
                 3130326 고덕 골목형상점가                       0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)    0.0000   

                                                       아파트_가격_1_억_미만_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                            
2023    1        3001491 이태원 관광특구                                  4.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구                    0.0000   
                 3001493 동대문패션타운 관광특구                              0.0000   
                 3001494 종로?청계 관광특구                                2.0000   
                 3001495 잠실 관광특구                                   9.0000   
...                                                                   ...   
                 3130323 둔촌역전통시장                                   0.0000   
                 3130324 길동복조리시장                                   0.0000   
                 3130325 명일전통시장                                    0.0000   
                 3130326 고덕 골목형상점가                                 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)              0.0000   

                                                       아파트_가격_1_억_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                         
2023    1        3001491 이태원 관광특구                              53.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구                 0.0000   
                 3001493 동대문패션타운 관광특구                          11.0000   
                 3001494 종로?청계 관광특구                            11.0000   
                 3001495 잠실 관광특구                               44.0000   
...                                                                ...   
                 3130323 둔촌역전통시장                                0.0000   
                 3130324 길동복조리시장                                0.0000   
                 3130325 명일전통시장                                 0.0000   
                 3130326 고덕 골목형상점가                              0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)           0.0000   

                                                       아파트_가격_2_억_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                         
2023    1        3001491 이태원 관광특구                              58.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구                 0.0000   
                 3001493 동대문패션타운 관광특구                           0.0000   
                 3001494 종로?청계 관광특구                            12.0000   
                 3001495 잠실 관광특구                               57.0000   
...                                                                ...   
                 3130323 둔촌역전통시장                                0.0000   
                 3130324 길동복조리시장                                0.0000   
                 3130325 명일전통시장                                 0.0000   
                 3130326 고덕 골목형상점가                              0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)           0.0000   

                                                       아파트_가격_3_억_세대_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                         
2023    1        3001491 이태원 관광특구                             110.0000   
                 3001492 명동 남대문 북

#### 5.(3) 상권배후지_점포

  MARKET_2022_LIST[6]

  MARKET_2023_LIST[5]

  ```
  SELECT
    AVERAGE(seoul.상권특성.개업_점포_수),
    AVERAGE(seoul.상권특성.유사_업종_점포_수),
    AVERAGE(seoul.상권특성.폐업_점포_수),
    AVERAGE(seoul.상권특성.점포_수),
    AVERAGE(seoul.상권특성.프랜차이즈_점포_수),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.개업_율),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.폐업_율),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [ ]:
display(MERGE_2022_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["서비스_업종_코드_명",
       "점포_수",
       "유사_업종_점포_수",
       "개업_점포_수",
       "폐업_점포_수",
       "프랜차이즈_점포_수"]].agg("mean"))

<ipython-input-119-58de832c413d>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  "프랜차이즈_점포_수"]].agg("mean"))


점포_수  유사_업종_점포_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                           
2022    1        2110001 이북5도청사                       1.7391      1.8696   
                 2110002 독립문역 1번                      2.5882      2.7941   
                 2110003 세검정초등학교                      3.2821      3.4359   
                 2110004 대신고등학교                       1.4000      1.5000   
                 2110005 세검정                          1.9268      2.0732   
...                                                      ...         ...   
        4        3130323 둔촌역전통시장                      0.0000      0.0000   
                 3130324 길동복조리시장                      0.0000      0.0000   
                 3130325 명일전통시장                       0.0000      0.0000   
                 3130326 고덕 골목형상점가                    0.0000      0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운) 0.0000      0.0000   

                                                       개업_점포_수  폐업_점포_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                          
2022    1        2110001 이북5도청사                         0.0000   0.0000   
                 2110002 독립문역 1번                        0.0882   0.0000   
                 2110003 세검정초등학교                        0.0256   0.0000   
                 2110004 대신고등학교                         0.0000   0.0000   
                 2110005 세검정                            0.0976   0.0732   
...                                                        ...      ...   
        4        3130323 둔촌역전통시장                        0.0000   0.0000   
                 3130324 길동복조리시장                        0.0000   0.0000   
                 3130325 명일전통시장                         0.0000   0.0000   
                 3130326 고덕 골목형상점가                      0.0000   0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)   0.0000   0.0000   

                                                       프랜차이즈_점포_수  
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                   
2022    1        2110001 이북5도청사                            0.1304  
                 2110002 독립문역 1번                           0.2059  
                 2110003 세검정초등학교                           0.1538  
                 2110004 대신고등학교                            0.1000  
                 2110005 세검정                               0.1463  
...                                                           ...  
        4        3130323 둔촌역전통시장                           0.0000  
                 3130324 길동복조리시장                           0.0000  
                 3130325 명일전통시장                            0.0000  
                 3130326 고덕 골목형상점가                         0.0000  
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)      0.0000  

[10792 rows x 5 columns]

In [ ]:
display(MERGE_2023_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["서비스_업종_코드_명",
       "점포_수",
       "유사_업종_점포_수",
       "개업_점포_수",
       "폐업_점포_수",
       "프랜차이즈_점포_수"]].agg("mean"))

<ipython-input-120-11d97917f30c>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  "프랜차이즈_점포_수"]].agg("mean"))


점포_수  유사_업종_점포_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                             
2023    1        3001491 이태원 관광특구                      19.9059     20.7765   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구        93.2921     99.5955   
                 3001493 동대문패션타운 관광특구                 138.5682    140.8523   
                 3001494 종로?청계 관광특구                    54.7253     57.1648   
                 3001495 잠실 관광특구                       25.3474     28.6105   
...                                                        ...         ...   
                 3130323 둔촌역전통시장                        0.0000      0.0000   
                 3130324 길동복조리시장                        0.0000      0.0000   
                 3130325 명일전통시장                         0.0000      0.0000   
                 3130326 고덕 골목형상점가                      0.0000      0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)   0.0000      0.0000   

                                                       개업_점포_수  폐업_점포_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                          
2023    1        3001491 이태원 관광특구                       0.4588   0.4235   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구         1.4382   2.1236   
                 3001493 동대문패션타운 관광특구                   0.6250   4.2727   
                 3001494 종로?청계 관광특구                     0.7033   1.2418   
                 3001495 잠실 관광특구                        0.4737   0.7263   
...                                                        ...      ...   
                 3130323 둔촌역전통시장                        0.0000   0.0000   
                 3130324 길동복조리시장                        0.0000   0.0000   
                 3130325 명일전통시장                         0.0000   0.0000   
                 3130326 고덕 골목형상점가                      0.0000   0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)   0.0000   0.0000   

                                                       프랜차이즈_점포_수  
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                   
2023    1        3001491 이태원 관광특구                          0.8706  
                 3001492 명동 남대문 북창동 다동 무교동 관광특구            6.3034  
                 3001493 동대문패션타운 관광특구                      2.2841  
                 3001494 종로?청계 관광특구                        2.4396  
                 3001495 잠실 관광특구                           3.2632  
...                                                           ...  
                 3130323 둔촌역전통시장                           0.0000  
                 3130324 길동복조리시장                           0.0000  
                 3130325 명일전통시장                            0.0000  
                 3130326 고덕 골목형상점가                         0.0000  
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)      0.0000  

[1650 rows x 5 columns]

#### 5.(4) 상권배후지_집객시설

  MARKET_2022_LIST[7]

  MARKET_2023_LIST[6]
    
  ```
  SELECT
    AVERAGE(seoul.상권특성.집객시설_수),
    AVERAGE(seoul.상권특성.관공서_수),
    AVERAGE(seoul.상권특성.은행_수),
    AVERAGE(seoul.상권특성.종합병원_수),
    AVERAGE(seoul.상권특성.일반_병원_수),
    AVERAGE(seoul.상권특성.약국_수),
    AVERAGE(seoul.상권특성.유치원_수),
    AVERAGE(seoul.상권특성.초등학교_수),
    AVERAGE(seoul.상권특성.고등학교_수),
    AVERAGE(seoul.상권특성.대학교_수),
    AVERAGE(seoul.상권특성.백화점_수),
    AVERAGE(seoul.상권특성.슈퍼마켓_수),
    AVERAGE(seoul.상권특성.숙박_시설_수),AVERAGE(seoul.상권특성.버스_터미널_수),
    AVERAGE(seoul.상권특성.지하철_역_수),
    AVERAGE(seoul.상권특성.버스_정거장_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [ ]:
display(MERGE_2022_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["집객시설_수",
       "관공서_수",
       "은행_수",
       "종합병원_수",
       "일반_병원_수",
       "약국_수",
       "유치원_수",
       "초등학교_수",
       "중학교_수",
       "고등학교_수",
       "대학교_수",
       "백화점_수",
       "슈퍼마켓_수",
       "극장_수",
       "숙박_시설_수",
       "공항_수",
       "철도_역_수",
       "버스_터미널_수",
       "지하철_역_수",
       "버스_정거장_수"]].agg("mean"))

집객시설_수  관공서_수   은행_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                               
2022    1        2110001 이북5도청사                        42.0000 2.0000 0.0000   
                 2110002 독립문역 1번                       57.0000 3.0000 1.0000   
                 2110003 세검정초등학교                      102.0000 4.0000 2.0000   
                 2110004 대신고등학교                        87.0000 6.0000 3.0000   
                 2110005 세검정                           85.0000 3.0000 2.0000   
...                                                        ...    ...    ...   
        4        3130323 둔촌역전통시장                        0.0000 0.0000 0.0000   
                 3130324 길동복조리시장                        0.0000 0.0000 0.0000   
                 3130325 명일전통시장                         0.0000 0.0000 0.0000   
                 3130326 고덕 골목형상점가                      0.0000 0.0000 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)   0.0000 0.0000 0.0000   

                                                       종합병원_수  일반_병원_수   약국_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                                
2022    1        2110001 이북5도청사                        0.0000   0.0000 2.0000   
                 2110002 독립문역 1번                       1.0000   0.0000 4.0000   
                 2110003 세검정초등학교                       0.0000   0.0000 3.0000   
                 2110004 대신고등학교                        0.0000   0.0000 5.0000   
                 2110005 세검정                           0.0000   0.0000 3.0000   
...                                                       ...      ...    ...   
        4        3130323 둔촌역전통시장                       0.0000   0.0000 0.0000   
                 3130324 길동복조리시장                       0.0000   0.0000 0.0000   
                 3130325 명일전통시장                        0.0000   0.0000 0.0000   
                 3130326 고덕 골목형상점가                     0.0000   0.0000 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)  0.0000   0.0000 0.0000   

                                                       유치원_수  초등학교_수  중학교_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                              
2022    1        2110001 이북5도청사                       0.0000  0.0000 0.0000   
                 2110002 독립문역 1번                      1.0000  1.0000 0.0000   
                 2110003 세검정초등학교                      1.0000  2.0000 1.0000   
                 2110004 대신고등학교                       1.0000  1.0000 0.0000   
                 2110005 세검정                          1.0000  2.0000 1.0000   
...                                                      ...     ...    ...   
        4        3130323 둔촌역전통시장                      0.0000  0.0000 0.0000   
                 3130324 길동복조리시장                      0.0000  0.0000 0.0000   
                 3130325 명일전통시장                       0.0000  0.0000 0.0000   
                 3130326 고덕 골목형상점가                    0.0000  0.0000 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운) 0.0000  0.0000 0.0000   

                                                       고등학교_수   대학교_수  백화점_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                               
2022    1        2110001 이북5도청사                        0.0000  0.0000 0.0000   
                 2110002 독립문역 1번                       1.0000  0.0000 0.0000   
                 2110003 세검정초등학교                       1.0000 15.0000 0.0000   
                 2110004 대신고등학교                        1.0000  0.0000 0.0000   
                 2110005 세검정                           1.0000 12.0000 0.0000   
...                                                       ...     ...    ...   
        4        3130323 둔촌역전통시장                       0.0000  0.0000 0.0000   
                 3130324 길동복조리시장                       0.0000  0.0000 0.0000   
                 3130325 명일전통시장                        0.0000  0.0000 0.0000   
                 3130326 고덕 골목형상점가                   

In [ ]:
display(MERGE_2023_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["집객시설_수",
       "관공서_수",
       "은행_수",
       "종합병원_수",
       "일반_병원_수",
       "약국_수",
       "유치원_수",
       "초등학교_수",
       "중학교_수",
       "고등학교_수",
       "대학교_수",
       "백화점_수",
       "슈퍼마켓_수",
       "극장_수",
       "숙박_시설_수",
       "공항_수",
       "철도_역_수",
       "버스_터미널_수",
       "지하철_역_수",
       "버스_정거장_수"]].agg("mean"))

집객시설_수  관공서_수   은행_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                              
2023    1        3001491 이태원 관광특구                      0.0000 0.0000 0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구        0.0000 0.0000 0.0000   
                 3001493 동대문패션타운 관광특구                  0.0000 0.0000 0.0000   
                 3001494 종로?청계 관광특구                    0.0000 0.0000 0.0000   
                 3001495 잠실 관광특구                       0.0000 0.0000 0.0000   
...                                                       ...    ...    ...   
                 3130323 둔촌역전통시장                       0.0000 0.0000 0.0000   
                 3130324 길동복조리시장                       0.0000 0.0000 0.0000   
                 3130325 명일전통시장                        0.0000 0.0000 0.0000   
                 3130326 고덕 골목형상점가                     0.0000 0.0000 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)  0.0000 0.0000 0.0000   

                                                       종합병원_수  일반_병원_수   약국_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                                
2023    1        3001491 이태원 관광특구                      0.0000   0.0000 0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구        0.0000   0.0000 0.0000   
                 3001493 동대문패션타운 관광특구                  0.0000   0.0000 0.0000   
                 3001494 종로?청계 관광특구                    0.0000   0.0000 0.0000   
                 3001495 잠실 관광특구                       0.0000   0.0000 0.0000   
...                                                       ...      ...    ...   
                 3130323 둔촌역전통시장                       0.0000   0.0000 0.0000   
                 3130324 길동복조리시장                       0.0000   0.0000 0.0000   
                 3130325 명일전통시장                        0.0000   0.0000 0.0000   
                 3130326 고덕 골목형상점가                     0.0000   0.0000 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)  0.0000   0.0000 0.0000   

                                                       유치원_수  초등학교_수  중학교_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                              
2023    1        3001491 이태원 관광특구                     0.0000  0.0000 0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구       0.0000  0.0000 0.0000   
                 3001493 동대문패션타운 관광특구                 0.0000  0.0000 0.0000   
                 3001494 종로?청계 관광특구                   0.0000  0.0000 0.0000   
                 3001495 잠실 관광특구                      0.0000  0.0000 0.0000   
...                                                      ...     ...    ...   
                 3130323 둔촌역전통시장                      0.0000  0.0000 0.0000   
                 3130324 길동복조리시장                      0.0000  0.0000 0.0000   
                 3130325 명일전통시장                       0.0000  0.0000 0.0000   
                 3130326 고덕 골목형상점가                    0.0000  0.0000 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운) 0.0000  0.0000 0.0000   

                                                       고등학교_수  대학교_수  백화점_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                              
2023    1        3001491 이태원 관광특구                      0.0000 0.0000 0.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구        0.0000 0.0000 0.0000   
                 3001493 동대문패션타운 관광특구                  0.0000 0.0000 0.0000   
                 3001494 종로?청계 관광특구                    0.0000 0.0000 0.0000   
                 3001495 잠실 관광특구                       0.0000 0.0000 0.0000   
...                                                       ...    ...    ...   
                 3130323 둔촌역전통시장                       0.0000 0.0000 0.0000   
                 3130324 길동복조리시장                       0.0000 0.0000 0.0000   
                 3130325 명일전통시장                        0.0000 0.0000 0.0000   
                 3130326 고덕 골목형상점가                     0.0000 0.0000 0.0000 

#### 5.(5) 상권_상주인구(상권-생활인구)

  MARKET_2022_LIST[11]

  MARKET_2022_LIST[10]

  ```
  SELECT
    AVERAGE(seoul.상권특성.연령대_10_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_20_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_30_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_40_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_50_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_60_이상_상주인구_수),
    AVERAGE(seoul.상권특성.총_가구_수),
    AVERAGE(seoul.상권특성.비_아파트_가구_수),
    AVERAGE(seoul.상권특성.아파트_가구_수),
    AVERAGE(seoul.상권특성.총_상주인구_수),
    AVERAGE(seoul.상권특성.남성_상주인구_수),
    AVERAGE(seoul.상권특성.여성_상주인구_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [ ]:
display(MERGE_2022_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["총_생활인구_수", "남성_생활인구_수", "여성_생활인구_수",
       "연령대_10_생활인구_수", "연령대_20_생활인구_수", "연령대_30_생활인구_수", "연령대_40_생활인구_수", "연령대_50_생활인구_수", "연령대_60_이상_생활인구_수",
       "시간대_1_생활인구_수", "시간대_2_생활인구_수", "시간대_3_생활인구_수", "시간대_4_생활인구_수", "시간대_5_생활인구_수", "시간대_6_생활인구_수",
       "월요일_생활인구_수", "화요일_생활인구_수", "수요일_생활인구_수", "목요일_생활인구_수", "금요일_생활인구_수", "토요일_생활인구_수", "일요일_생활인구_수"]].agg("mean"))

총_생활인구_수   남성_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                                
2022    1        2110001 이북5도청사                       148832.0000  66007.0000   
                 2110002 독립문역 1번                      302828.0000 137022.0000   
                 2110003 세검정초등학교                      403155.0000 181227.0000   
                 2110004 대신고등학교                       369100.0000 170943.0000   
                 2110005 세검정                          173118.0000  75228.0000   
...                                                           ...         ...   
        4        3130323 둔촌역전통시장                           0.0000      0.0000   
                 3130324 길동복조리시장                           0.0000      0.0000   
                 3130325 명일전통시장                            0.0000      0.0000   
                 3130326 고덕 골목형상점가                         0.0000      0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)      0.0000      0.0000   

                                                        여성_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                    
2022    1        2110001 이북5도청사                        82825.0000   
                 2110002 독립문역 1번                      165805.0000   
                 2110003 세검정초등학교                      221927.0000   
                 2110004 대신고등학교                       198156.0000   
                 2110005 세검정                           97890.0000   
...                                                           ...   
        4        3130323 둔촌역전통시장                           0.0000   
                 3130324 길동복조리시장                           0.0000   
                 3130325 명일전통시장                            0.0000   
                 3130326 고덕 골목형상점가                         0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)      0.0000   

                                                       연령대_10_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2022    1        2110001 이북5도청사                           26170.0000   
                 2110002 독립문역 1번                          57664.0000   
                 2110003 세검정초등학교                          74048.0000   
                 2110004 대신고등학교                           71279.0000   
                 2110005 세검정                              29350.0000   
...                                                              ...   
        4        3130323 둔촌역전통시장                              0.0000   
                 3130324 길동복조리시장                              0.0000   
                 3130325 명일전통시장                               0.0000   
                 3130326 고덕 골목형상점가                            0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)         0.0000   

                                                       연령대_20_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2022    1        2110001 이북5도청사                           16060.0000   
                 2110002 독립문역 1번                          36328.0000   
                 2110003 세검정초등학교                          45819.0000   
                 2110004 대신고등학교                           42912.0000   
                 2110005 세검정                              17753.0000   
...                                                              ...   
        4        3130323 둔촌역전통시장                              0.0000   
                 3130324 길동복조리시장                              0.0000   
                 3130325 명일전통시장                               0.0000   
                 3130326 고덕 골목형상점가                            0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)         0.0000   

                                                       연령대_30_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2022    1        2110001 이북5도청사                           14849.0000   
               

In [ ]:
display(MERGE_2023_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["총_생활인구_수", "남성_생활인구_수", "여성_생활인구_수",
       "연령대_10_생활인구_수", "연령대_20_생활인구_수", "연령대_30_생활인구_수", "연령대_40_생활인구_수", "연령대_50_생활인구_수", "연령대_60_이상_생활인구_수",
       "시간대_1_생활인구_수", "시간대_2_생활인구_수", "시간대_3_생활인구_수", "시간대_4_생활인구_수", "시간대_5_생활인구_수", "시간대_6_생활인구_수",
       "월요일_생활인구_수", "화요일_생활인구_수", "수요일_생활인구_수", "목요일_생활인구_수", "금요일_생활인구_수", "토요일_생활인구_수", "일요일_생활인구_수"]].agg("mean"))

총_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                     
2023    1        3001491 이태원 관광특구                     2014294.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구       7334315.0000   
                 3001493 동대문패션타운 관광특구                 3352203.0000   
                 3001494 종로?청계 관광특구                   8502972.0000   
                 3001495 잠실 관광특구                      4186941.0000   
...                                                            ...   
                 3130323 둔촌역전통시장                            0.0000   
                 3130324 길동복조리시장                            0.0000   
                 3130325 명일전통시장                             0.0000   
                 3130326 고덕 골목형상점가                          0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)       0.0000   

                                                         남성_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                     
2023    1        3001491 이태원 관광특구                     1003011.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구       3588578.0000   
                 3001493 동대문패션타운 관광특구                 1590289.0000   
                 3001494 종로?청계 관광특구                   4590859.0000   
                 3001495 잠실 관광특구                      2027264.0000   
...                                                            ...   
                 3130323 둔촌역전통시장                            0.0000   
                 3130324 길동복조리시장                            0.0000   
                 3130325 명일전통시장                             0.0000   
                 3130326 고덕 골목형상점가                          0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)       0.0000   

                                                         여성_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                     
2023    1        3001491 이태원 관광특구                     1011283.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구       3745738.0000   
                 3001493 동대문패션타운 관광특구                 1761913.0000   
                 3001494 종로?청계 관광특구                   3912112.0000   
                 3001495 잠실 관광특구                      2159678.0000   
...                                                            ...   
                 3130323 둔촌역전통시장                            0.0000   
                 3130324 길동복조리시장                            0.0000   
                 3130325 명일전통시장                             0.0000   
                 3130326 고덕 골목형상점가                          0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)       0.0000   

                                                       연령대_10_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2023    1        3001491 이태원 관광특구                        133431.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구          317233.0000   
                 3001493 동대문패션타운 관광특구                    182026.0000   
                 3001494 종로?청계 관광특구                      351982.0000   
                 3001495 잠실 관광특구                         488085.0000   
...                                                              ...   
                 3130323 둔촌역전통시장                              0.0000   
                 3130324 길동복조리시장                              0.0000   
                 3130325 명일전통시장                               0.0000   
                 3130326 고덕 골목형상점가                            0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)         0.0000   

                                                       연령대_20_생활인구_수  \
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                       
2023    1        3001491 이태원 관광특구                        459329.0000   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구         1266022.0000   
                 3001493 동대문패션타운 관광특구                    596012.0000   
                 3001494 종로?

#### 5.(6) 상권_점포(행정동별 상권변화지표)

  ['상권축소', '정체', '다이나믹', '상권확장']
  
  MARKET_2022_LIST[13]

  MARKET_2023_LIST[12]

In [ ]:
display(MERGE_2022_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["개업_율", "폐업_률"]].agg(["mean", "std"]))

개업_율           폐업_률  \
                                                        mean     std   mean   
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                              
2022    1        2110001 이북5도청사                       0.0000  0.0000 0.0000   
                 2110002 독립문역 1번                      5.1471 19.2466 0.0000   
                 2110003 세검정초등학교                      0.5128  3.2026 0.0000   
                 2110004 대신고등학교                       0.0000  0.0000 0.0000   
                 2110005 세검정                          1.8293  6.8297 0.4146   
...                                                      ...     ...    ...   
        4        3130323 둔촌역전통시장                      0.0000     NaN 0.0000   
                 3130324 길동복조리시장                      0.0000     NaN 0.0000   
                 3130325 명일전통시장                       0.0000     NaN 0.0000   
                 3130326 고덕 골목형상점가                    0.0000     NaN 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운) 0.0000  0.0000 0.0000   

                                                              
                                                         std  
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                              
2022    1        2110001 이북5도청사                       0.0000  
                 2110002 독립문역 1번                      0.0000  
                 2110003 세검정초등학교                      0.0000  
                 2110004 대신고등학교                       0.0000  
                 2110005 세검정                          2.6550  
...                                                      ...  
        4        3130323 둔촌역전통시장                         NaN  
                 3130324 길동복조리시장                         NaN  
                 3130325 명일전통시장                          NaN  
                 3130326 고덕 골목형상점가                       NaN  
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운) 0.0000  

[10792 rows x 4 columns]

In [ ]:
display(MERGE_2023_DATA.fillna(0).groupby(
    ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"]
    )[["개업_율", "폐업_률"]].agg(["mean", "std"]))

개업_율           폐업_률  \
                                                        mean     std   mean   
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                                              
2023    1        3001491 이태원 관광특구                     4.0588 15.8740 3.0824   
                 3001492 명동 남대문 북창동 다동 무교동 관광특구       1.3820  3.2532 1.9213   
                 3001493 동대문패션타운 관광특구                 1.0909  3.6783 1.4318   
                 3001494 종로?청계 관광특구                   1.3736  2.6922 2.3626   
                 3001495 잠실 관광특구                      1.5474  3.3983 2.1474   
...                                                      ...     ...    ...   
                 3130323 둔촌역전통시장                      0.0000     NaN 0.0000   
                 3130324 길동복조리시장                      0.0000     NaN 0.0000   
                 3130325 명일전통시장                       0.0000     NaN 0.0000   
                 3130326 고덕 골목형상점가                    0.0000     NaN 0.0000   
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운) 0.0000  0.0000 0.0000   

                                                               
                                                          std  
기준_년_코드 기준_분기_코드 상권_코드   상권_코드_명                               
2023    1        3001491 이태원 관광특구                     15.3384  
                 3001492 명동 남대문 북창동 다동 무교동 관광특구        3.2591  
                 3001493 동대문패션타운 관광특구                  4.4869  
                 3001494 종로?청계 관광특구                    6.0167  
                 3001495 잠실 관광특구                       4.6562  
...                                                       ...  
                 3130323 둔촌역전통시장                          NaN  
                 3130324 길동복조리시장                          NaN  
                 3130325 명일전통시장                           NaN  
                 3130326 고덕 골목형상점가                        NaN  
                 3130327 평화시장(남평화시장, 제일평화시장, 신평화패션타운)  0.0000  

[1650 rows x 4 columns]

___

### 6. 분기별 상권별 상권특성 공공데이터 병합

In [ ]:
MERGE_2022_COLUMNS = MERGE_2022_DATA.columns.tolist()

for item in ["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명", "상권_구분_코드",  '기준_년월_코드',
             '엑스좌표_값', '와이좌표_값', '시군구_코드', '행정동_코드', '기준_월_코드', '법정동_코드', 'STDG_EMD_CD', '행정동_코드_명']:
  MERGE_2022_COLUMNS.remove(item)

In [ ]:
MARKET_2022_GROUPED = MERGE_2022_DATA.fillna(0).groupby(["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"])[MERGE_2022_COLUMNS].agg("mean")
MARKET_2022_RATE = MERGE_2022_DATA.fillna(0).groupby(["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"])[["개업_율", "폐업_률"]].agg(["mean", "std"])

<ipython-input-148-d51610500405>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  MARKET_2022_GROUPED = MERGE_2022_DATA.fillna(0).groupby(["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"])[MERGE_2022_COLUMNS].agg("mean")


In [ ]:
MARKET_2023_GROUPED = MERGE_2023_DATA.fillna(0).groupby(["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"])[MERGE_2022_COLUMNS].agg("mean")
MARKET_2023_RATE = MERGE_2023_DATA.fillna(0).groupby(["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"])[["개업_율", "폐업_률"]].agg(["mean", "std"])

<ipython-input-150-14c4ceed6e20>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  MARKET_2023_GROUPED = MERGE_2023_DATA.fillna(0).groupby(["기준_년_코드", "기준_분기_코드", "상권_코드", "상권_코드_명"])[MERGE_2022_COLUMNS].agg("mean")


In [ ]:
# MARKET_2022_GROUPED.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_2022_GROUPED.csv", index = True)
# MARKET_2023_GROUPED.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_2023_GROUPED.csv", index = True)

### 7. 분기별 상권데이터를 소호 신용데이터를 활용하여 월별로 확장(계절성 지수)

In [ ]:
MARKET_2022_GROUPED = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2022_GROUPED.csv",
    unicode = "utf-8", columnTF = False)

In [ ]:
MARKET_2023_GROUPED = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2023_GROUPED.csv",
    unicode = "utf-8", columnTF = False)

In [ ]:
"""
MARKET_20221Q_GROUPED = MARKET_2022_GROUPED[MARKET_2022_GROUPED["기준_분기_코드"] == 1]
MARKET_20221Q_GROUPED["2022년도_상권_코드"] = MARKET_20221Q_GROUPED["상권_코드"].apply(lambda data: True if str(data)[0] == "2" else False)
Counter(MARKET_20221Q_GROUPED["2022년도_상권_코드"])
  # Counter({True: 1048, False: 1650})
MARKET_20221Q_GROUPED[MARKET_20221Q_GROUPED["2022년도_상권_코드"] == False]

MARKET_2022_GROUPED["2022년도_상권_코드"] = MARKET_2022_GROUPED["상권_코드"].apply(lambda data: True if str(data)[0] == "2" else False)
MARKET_2022_GROUPED = MARKET_2022_GROUPED[MARKET_2022_GROUPED["2022년도_상권_코드"] == True]
MARKET_2022_GROUPED.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_2022_GROUPED.csv", index = False)
"""

In [ ]:
KCD_2022_2023.drop("Unnamed: 0", axis = 1, inplace = True)
KCD_2022_2023.fillna(0)
KCD_상권코드 = KCD_2022_2023["상권코드"].unique().tolist()

In [ ]:
KCD_2022_2023

,Unnamed: 0,기준일자,상권_식별자,상권코드,상권명,외식업종_중분류,소호사업장_수,경영위기사업장_총합,프랜차이즈사업장_총합,소상공인연령_평균,...,부가가치세_변동계수,부가세차감전영업이익_변동계수,사업장방문고객수_변동계수,사업장방문신규고객수_변동계수,사업장임대면적_변동계수,월임대료_변동계수,임대보증금_변동계수,손익분기점매출액_변동계수,객단가_변동계수,신규고객단가_변동계수
0,0,202201,DMC(디지털미디어시티)_카페,2120098.0000,DMC(디지털미디어시티),카페,13.0000,0.0000,3.0000,39.5846,...,-0.6996,1.1789,0.7065,0.4988,0.4128,0.5341,0.5648,-0.5301,0.7058,0.9996
1,1,202201,가락시장역_한식,2120234.0000,가락시장역,한식,5.0000,0.0000,3.0000,44.9400,...,-0.7722,1.1607,1.0395,1.0085,0.5250,0.5646,0.5735,0.4745,0.9883,1.0187
2,2,202201,가로수길_술집,2120186.0000,가로수길,술집,7.0000,0.0000,1.0000,42.3571,...,-1.3127,1.8294,1.7833,1.8097,0.5326,1.0291,1.1673,0.7954,0.8118,0.8000
3,3,202201,가로수길_일식,2120186.0000,가로수길,일식,7.0000,0.0000,1.0000,34.6571,...,-0.6643,0.4953,0.8916,0.8940,0.6296,0.3630,0.4541,0.3542,0.5502,0.5487
4,4,202201,가로수길_한식,2120186.0000,가로수길,한식,7.0000,0.0000,2.0000,41.9143,...,-0.5937,0.6890,0.7941,0.7777,0.3451,0.2461,0.3202,0.2583,0.7582,0.7741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,176,202303,홍대입구역(홍대)_한식,2120103.0000,홍대입구역(홍대),한식,28.0000,3.0000,2.0000,37.7571,...,-0.9489,1.1860,0.7307,0.7528,0.8892,0.6365,0.7507,0.7577,1.2670,1.2298
2688,177,202303,선릉역_카페,NaN,NaN,NaN,6.0000,1.0000,1.0000,35.3167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2689,178,202303,"신촌역(신촌역, 신촌로터리)_카페",NaN,NaN,NaN,5.0000,0.0000,1.0000,42.8600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2690,179,202303,"총신대입구역(이수, 총신대)_카페",NaN,NaN,NaN,5.0000,2.0000,2.0000,32.8800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
MARKET_2022_GROUPED

,기준_년_코드,기준_분기_코드,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,...,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,2022년도_상권_코드
0,2022,1,2110001,이북5도청사,4305934.0000,8.0000,520427635.0000,129126028.0000,64853632.0000,37122367.0000,...,21071.0000,20786.0000,20856.0000,21802.0000,22313.0000,0.0000,0.0000,0.0000,0.0000,True
1,2022,1,2110002,독립문역 1번,3463885.0000,7.0000,232465205.0000,54998987.0000,28932090.0000,17084860.0000,...,43501.0000,43530.0000,43251.0000,42989.0000,42471.0000,0.0000,0.0000,0.0000,0.0000,True
2,2022,1,2110003,세검정초등학교,3548020.0000,7.0000,1349699230.0000,336704220.0000,166150296.0000,98720958.0000,...,57667.0000,57611.0000,56583.0000,58411.0000,59445.0000,0.0000,0.0000,0.0000,0.0000,True
3,2022,1,2110004,대신고등학교,2644953.0000,6.0000,712772785.0000,205649728.0000,84170011.0000,51668351.0000,...,52107.0000,52086.0000,52161.0000,53644.0000,54136.0000,0.0000,0.0000,0.0000,0.0000,True
4,2022,1,2110005,세검정,3703145.0000,7.0000,884701599.0000,221446948.0000,109882417.0000,64738346.0000,...,25036.0000,24905.0000,24807.0000,24531.0000,24132.0000,0.0000,0.0000,0.0000,0.0000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,2022,4,2111085,묘곡초등학교(고덕아이파크아파트),2558553.0000,6.0000,5007325425.0000,1435306317.0000,590981199.0000,343018095.0000,...,100413.0000,99871.0000,97569.0000,102384.0000,104306.0000,0.0000,0.0000,0.0000,0.0000,True
4188,2022,4,2111086,한영중고,2851749.0000,6.0000,1264810182.0000,350821529.0000,149392590.0000,83087315.0000,...,52172.0000,52580.0000,52837.0000,49719.0000,47718.0000,0.0000,0.0000,0.0000,0.0000,True
4189,2022,4,2111087,고덕중학교(고덕2동주민센터),2493823.0000,6.0000,995226881.0000,307190926.0000,115234826.0000,64298750.0000,...,31221.0000,31050.0000,30171.0000,27473.0000,27081.0000,0.0000,0.0000,0.0000,0.0000,True
4190,2022,4,2111088,고덕동성당,2369104.0000,5.0000,941528487.0000,283232544.0000,109891194.0000,63370941.0000,...,58331.0000,59270.0000,59084.0000,58517.0000,58130.0000,0.0000,0.0000,0.0000,0.0000,True


In [ ]:
KCD_COLUMNS = KCD_2022_2023.columns.tolist()

In [ ]:
for column_name in ['Unnamed: 0','기준일자','상권_식별자','상권코드','상권명','외식업종_중분류','소상공인연령_평균','소상공인업력_평균']:
    KCD_COLUMNS.remove(column_name)

In [ ]:
print(KCD_COLUMNS, end = "")

['소호사업장_수', '경영위기사업장_총합', '프랜차이즈사업장_총합', '정규고용인원_평균', '사업장방문고객수_평균', '사업장방문신규고객수_평균', '매출액_변동계수', '카드매출액_변동계수', '배달매출액_변동계수', '주말카드매출액_변동계수', '주말배달매출액_변동계수', '매입액_변동계수', '매출총이익_변동계수', '부가가치세_변동계수', '부가세차감전영업이익_변동계수', '사업장방문고객수_변동계수', '사업장방문신규고객수_변동계수', '사업장임대면적_변동계수', '월임대료_변동계수', '임대보증금_변동계수', '손익분기점매출액_변동계수', '객단가_변동계수', '신규고객단가_변동계수']

In [ ]:
SEASONAL_PATTERN = KCD_2022_2023.groupby(["기준일자"])[KCD_COLUMNS].mean()

In [ ]:
SEASONAL_PATTERN = SEASONAL_PATTERN / SEASONAL_PATTERN.mean()

In [ ]:
pd.set_option("display.max_column", None)
SEASONAL_PATTERN

,소호사업장_수,경영위기사업장_총합,프랜차이즈사업장_총합,정규고용인원_평균,사업장방문고객수_평균,사업장방문신규고객수_평균,매출액_변동계수,카드매출액_변동계수,배달매출액_변동계수,주말카드매출액_변동계수,주말배달매출액_변동계수,매입액_변동계수,매출총이익_변동계수,부가가치세_변동계수,부가세차감전영업이익_변동계수,사업장방문고객수_변동계수,사업장방문신규고객수_변동계수,사업장임대면적_변동계수,월임대료_변동계수,임대보증금_변동계수,손익분기점매출액_변동계수,객단가_변동계수,신규고객단가_변동계수
기준일자,,,,,,,,,,,,,,,,,,,,,,,
202201,0.9528,0.0000,1.0180,0.9591,0.8503,0.8241,1.0216,1.0186,-0.0013,1.0336,-0.0004,0.9594,1.1218,0.8633,1.1300,1.0074,1.0347,0.9881,0.9942,0.9817,3.5917,1.0059,0.9720
202202,0.9538,1.4188,1.0061,0.9671,0.7280,0.6905,1.0428,1.0301,-0.0009,1.0220,0.0016,1.0660,1.1044,0.9850,1.3460,1.0239,1.0477,0.9952,0.9982,0.9847,15.2345,1.0160,0.9852
202203,0.9624,1.2986,1.0286,0.9714,0.8883,0.8567,1.0189,1.0144,0.0085,1.0197,-0.0001,0.9752,1.2202,0.8637,1.1547,1.0111,1.0222,0.9950,0.9973,0.9898,3.3060,1.0119,1.0034
202204,0.9555,1.0421,1.0118,0.9759,1.0398,1.0437,0.9799,0.9868,0.0071,0.9621,0.0005,0.9575,1.0251,0.7983,0.6626,0.9819,0.9882,0.9896,0.9936,0.9928,1.7315,0.9956,0.9876
202205,0.9694,0.8377,1.0061,0.9828,1.1280,1.1568,0.9956,1.0009,0.0143,0.9883,0.0005,0.9176,0.9729,2.3832,0.9948,0.9962,1.0019,1.0055,0.9963,1.0012,1.3886,0.9960,0.9923
202206,0.9776,1.0776,1.0111,0.9744,1.0538,1.0681,0.7842,0.7429,12.9139,0.8169,11.9954,0.9526,0.7913,0.7661,0.8397,0.7756,0.7783,0.9942,0.9914,0.9945,5.7873,1.0378,1.0346
202207,0.9946,1.0053,1.0000,0.9692,1.0902,1.1142,1.0038,1.0054,-0.0078,1.0000,NaN,0.9327,0.9907,0.8585,1.0310,1.0093,1.0124,0.9947,0.9986,0.9868,2.4555,0.9845,0.9875
202208,1.0081,1.0287,1.0110,0.9781,1.0373,1.0416,1.0142,1.0153,-0.0033,1.0169,0.0007,0.8904,0.9955,0.9200,0.6941,1.0143,1.0144,0.9950,1.0007,1.0318,1.9408,0.9874,0.9984
202209,1.0265,1.0889,1.0163,0.9851,1.0248,1.0257,1.0007,1.0047,-0.0022,1.0209,0.0011,0.9506,1.0406,0.5190,1.2496,1.0190,1.0038,1.0126,1.0069,1.0057,4.3873,0.9746,0.9977


In [ ]:
MARKET_2022_GROUPED.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_코드', '상권_코드_명', '월_평균_소득_금액', '소득_구간_코드',
       '지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액',
       '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액',
       '아파트_단지_수', '아파트_가격_1_억_미만_세대_수', '아파트_가격_1_억_세대_수', '아파트_가격_2_억_세대_수',
       '아파트_가격_3_억_세대_수', '아파트_가격_4_억_세대_수', '아파트_가격_5_억_세대_수',
       '아파트_가격_6_억_이상_세대_수', '점포_수', '유사_업종_점포_수', '개업_율', '개업_점포_수', '폐업_률',
       '폐업_점포_수', '프랜차이즈_점포_수', '집객시설_수', '관공서_수', '은행_수', '종합병원_수', '일반_병원_수',
       '약국_수', '유치원_수', '초등학교_수', '중학교_수', '고등학교_수', '대학교_수', '백화점_수',
       '슈퍼마켓_수', '극장_수', '숙박_시설_수', '공항_수', '철도_역_수', '버스_터미널_수', '지하철_역_수',
       '버스_정거장_수', '총_생활인구_수', '남성_생활인구_수', '여성_생활인구_수', '연령대_10_생활인구_수',
       '연령대_20_생활인구_수', '연령대_30_생활인구_수', '연령대_40_생활인구_수', '연령대_50_생활인구_수',
       '연령대_60_이상_생활인구_수', '시간대_1_생활인구_수', '시간대_2_생활인구_수', '시간대_3_생활인구_수',
       '시간대_4_생활인구_수', '시간대_5_생활인구_수', '시간대_6_생활인구_수', '월요일_생활인구_수',
       '화요일_생활인구_수', '수요일_생활인구_수',